In [1]:
import itertools
import logging
from pathlib import Path
import numba as nb

import awkward as ak
import click
import h5py
import numpy as np
import vector

#from src.data.cms.convert_to_h5 import MIN_JETS, N_JETS, N_FJETS

vector.register_awkward()

logging.basicConfig(level=logging.INFO)

In [2]:
HIGGS_MASS = 125.0
MIN_JETS = 6
N_JETS = 10

In [3]:
# a function that loads jets from hhh_test.h5
def load_jets(in_file):
    # load jets from the h5
    pt = ak.Array(in_file["INPUTS"]["Jets"]["pt"])
    eta = ak.Array(in_file["INPUTS"]["Jets"]["eta"])
    phi = ak.Array(in_file["INPUTS"]["Jets"]["phi"])
    btag = ak.Array(in_file["INPUTS"]["Jets"]["btag"])
    mass = ak.Array(in_file["INPUTS"]["Jets"]["mass"])
    mask = ak.Array(in_file["INPUTS"]["Jets"]["MASK"])

    jets = ak.zip(
        {
            "pt": pt,
            "eta": eta,
            "phi": phi,
            "btag": btag,
            "mass": mass,
            "mask": mask
        },
        with_name="Momentum4D",
    )
    
    return jets

In [4]:
JET_ASSIGNMENTS = {}
for nj in range(MIN_JETS, N_JETS + 1):
    a = list(itertools.combinations(range(nj), 2))
    b = np.array([(i, j, k) for i, j, k in itertools.combinations(a, 3) if len(set(i + j + k)) == MIN_JETS])
    JET_ASSIGNMENTS[nj] = b


In [5]:
in_file = "//Users/billyli/UCSD/hhh/reports/bv2/hhh_test.h5"
in_file = h5py.File(in_file)
jets = load_jets(in_file)

In [6]:
nj = 6
mjj = (jets[:, JET_ASSIGNMENTS[nj][:, :, 0]] + jets[:, JET_ASSIGNMENTS[nj][:, :, 1]]).mass
chi2 = ak.sum(np.square(mjj - HIGGS_MASS), axis=-1)
chi2_argmin = ak.argmin(chi2, axis=-1)

In [7]:
chi2_argmin = chi2_argmin.to_numpy(allow_missing=False).reshape(-1, 1)

In [8]:
chi2 = ak.to_numpy(chi2, allow_missing=False)

In [9]:
chi2_sum = np.take_along_axis(chi2, chi2_argmin, axis=-1)

array([[  872.79700636],
       [ 9803.28624733],
       [35882.93413304],
       ...,
       [ 2491.51387976],
       [49792.71390584],
       [13780.4842438 ]])

In [11]:
chi2[2,chi2_argmin[2]]

array([35882.93413304])